In [26]:
import pandas as pd
import numpy as np
from random import randint
import yaml
import tabula
import os

In [27]:
mak23 = tabula.read_pdf("./examples/Mak2023-2024.pdf", pages="all")

In [30]:
with open('./Notebooks/preprocessingConfig.yml', 'r') as file:
    config = yaml.safe_load(file)

In [31]:
columnNames = config['general']['columnNames']
columnNames2 = config['general']['columnNames2']

In [32]:
def instTest(x):
    try: 
        if isinstance(float(x), float):
            return True
    except ValueError:
        return False

In [33]:
# Function to extract data from data2
def dataProcess(data, n):
#     if n >= 1:
    studentData = data[n].split()
    # print("Student Data: ", studentData)
    index_no = studentData[0]
    name = " ".join(studentData[1:-7])[:-1]
    gender = studentData[-8][-1]
    UACE_YEAR = int(studentData[-7])
    DistrictCode = int(studentData[-6])
    District = studentData[-5]
    CourseCode = studentData[-4]
    Course = ' '.join(studentData[-3:])
    return [index_no, name, gender, UACE_YEAR,
            DistrictCode, District, CourseCode, Course]

# Get studentData
def getStudentData(data):
    studentList = []
    for x in range(len(data)):
    #     if i > 0:
        student = dataProcess(data, x)
        studentList.append(student)
    # print(studentList)
    return studentList

# final df
def getFinalDf(pdf_read):
    df = pd.DataFrame(columns=columnNames2, data=[])
    for i in range(len(pdf_read)):
        df = pd.concat([df, getDf(i)], ignore_index=True)
    return df

In [34]:
def getDf(i):
    # i = randint(0, len(mak23)-1)
    # i = 0
    # print(i)

    # First column is a common to all, drop it
    df = mak23[i]
    # print(df.head(5))
    
    columns = df.columns
    # print("columns: ", columns)
    df = df.drop(columns[0], axis=1)
    # print(df.head(10))

    # Drop columns with nan values
    # print("columns before dropping nan-columns: ", df.shape[1])
    df = df.dropna(axis=1,
                   # thresh=3,
                   how="all",
                  )
    # print("columns after dropping: ", df.shape[1])

    # drop rows with nan if they exist
    # print("Rows before dropping: ", df.shape[0])
    df = df.dropna(axis=0,
                   # how="any",
                   thresh=3,
                  )

    # Drop rows for which Tot_wt is not float
    df = df[df[columns[-1]] != "Tot_wt"]
    # print("Weights: ", df[columns[-1]])
    # print(df.shape)
    columns = df.columns
    # print(columns)
    if len(columns) == 3:        
        
        # print(df.head(2))
        data = df[columns[1]].tolist()
        # if i == 0:
        #     data = data[1:]

        # print(data[:4])
        # data = df[columns.tolist()[1]].split()
        # print(df[df.columns[-1]])

        appData = df[df.columns[0]].tolist()
        # print(appData)
        Tot_weights = df[df.columns[-1]].tolist()
        # print("Length of Tot_weights: ", len(Tot_weights))
        # print("Samples of Tot_weights")
        # print(Tot_weights)
        temp = getStudentData(data)
        # print(temp)
        df = pd.DataFrame(columns=columnNames, data=temp)
        # print("Number of columns: ", df.shape)
        # print("Shape of temp: ", len(temp))
        # insert columns at last position
        df.insert(loc=len(df.columns),
                  column="TOTAL WEIGHT",
                  value=Tot_weights,
                 )
        df.insert(loc=0,
                  column="APPLICATION NO",
                  value=appData,
                 )
        # insert Appplication number column
        appNoData = mak23[i].loc[0]
        return df
    if len(df.columns) == 10:
        df = df[df[columns[-1]] != "Tot_wt"]
        # Rename columns
        columns = df.columns
        df = df.rename(columns={columns[x]:columnNames2[x] for x in range(len(columns))})
        return df

In [43]:
# Save dataframe
def save_finalDf(df):
    # current working directory
    current_path = os.getcwd()
    print(current_path)
    filepath = os.path.abspath(os.path.join(current_path, 'Data'))
    filename = "Private.csv"

    print(filepath)
    saved_file = df.to_csv(os.path.join(filepath, filename), index=False)
    return saved_file

In [44]:
save_finalDf(getFinalDf(mak23))

D:\Novel\Data\Python\DataAnalysis\Makerere2023-2024
D:\Novel\Data\Python\DataAnalysis\Makerere2023-2024\Data


In [47]:
df = pd.read_csv("./Data/Private.csv")

In [48]:
df

,APPLICATION NO,INDEX NO,NAME,GENDER,UACE YEAR,CODE,DISTRICT,COURSE CODE,COURSE NAME,TOTAL WEIGHT
0,23APF412304454141015,U0391/673,LUKABWE DOUGLAS MARK,M,2022.0,42.0,MUKONO,ACC,BSC. Accounting (EVE),42.2
1,23APF345674424112360,U1249/532,LULE ISAAC,M,2022.0,40.0,MPIGI,ACC,BSC. Accounting (EVE),41.1
2,23APF1361730063122187,U0459/676,HAULAH SULAIT BIIRA,F,2022.0,21.0,KASESE,ACC,BSC. Accounting (EVE),40.6
3,23APF519647933132646,U1611/887,NAMBALYE EVELYNE,F,2022.0,33.0,MASAKA,ACC,BSC. Accounting (EVE),38.8
4,23APF1481620160129238,U0083/555,KEMIGISHA RUSIA,F,2022.0,65.0,IBANDA,ACC,BSC. Accounting (EVE),38.4
...,...,...,...,...,...,...,...,...,...,...
14910,23APF727648568141557,U0017/548,KIZZA VANESSA ITRONCE,F,2022.0,17.0,KAMULI,VET,B. Veterinary Medicine,41.8
14911,23APF963075058135059,U0033/581,OTIM DANIEL,M,2022.0,31.0,LIRA,VET,B. Veterinary Medicine,41.7
14912,23APF1618205309130850,U0077/736,MUKETI JOVIA,F,2022.0,74.0,MARACHA,VET,B. Veterinary Medicine,41.6
14913,23APF575513287117711,U0077/564,AWOR REBECCA,F,2022.0,31.0,LIRA,VET,B. Veterinary Medicine,40.4
